In [4]:
from model import Network
from data import load_cifar10_dataloaders, WhiteningTransformation
import torch
import torchvision.transforms as transforms
import torch.nn as nn
from criterion import ReconstructImageFromFCLoss
import numpy as np
from metrics import lpips_gpu
import optuna

In [5]:
def objective(trial):
    # load to model
    model_config = './model_config/fc2_cocktail_party_cifar10_instance.json'
    checkpoint_path = './checkpoints/122123_fc2_cocktail_party_cifar10_pretraining_wout_bias_wout_normalization.pth'
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = Network(model_config)
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['state_dict'])
    model = model.to(device)

    # get val loader
    batch_size = 16
    data_path = './data'
    transform = transforms.Compose([
        transforms.ToTensor(),
    ])

    # reproducibility
    random_seed = 2024

    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    np.random.seed(random_seed)

    _, val_dataloader = load_cifar10_dataloaders(data_path, batch_size, transform)
    selected_val_batch_data, selected_val_batch_label = next(iter(val_dataloader))
    selected_val_batch_data = selected_val_batch_data.to(device)
    selected_val_batch_label = selected_val_batch_label.to(device)
    
    # receiving gradients
    model.zero_grad()
    criterion = nn.CrossEntropyLoss()
    output = model(selected_val_batch_data.reshape(batch_size, -1))
    loss = criterion(output, selected_val_batch_label)
    loss.backward()
    gradient_of_layers = []
    for param in model.parameters():
        gradient_of_layers.append(param.grad.data.clone().to('cpu'))
    print([x.size() for x in gradient_of_layers])
    
    whitening_transform = WhiteningTransformation()
    whitened_gradient = torch.from_numpy(whitening_transform.transform(gradient_of_layers[0].detach().numpy().T)).to(torch.float32).T
    whitened_gradient = whitened_gradient.to(device)
    
    # criterion output testing
    unmixing_matrix = torch.rand((selected_val_batch_data.size(0), gradient_of_layers[0].size(0)), requires_grad=True, device=device)
    t_param = trial.suggest_float('t_param', 0.00001, 10)
    total_variance_loss_param = trial.suggest_float('total_variance_loss_param', 0.00001, 10)
    mutual_independence_loss_param = trial.suggest_float('mutual_independence_loss_param', 0.00001, 10)
    reconstruction_loss = ReconstructImageFromFCLoss(32, 32, 3, t_param, total_variance_loss_param, mutual_independence_loss_param)
    optimizer = torch.optim.Adam([unmixing_matrix])
    
    for iter_idx in range(25000):
        optimizer.zero_grad()
        # out_score, non_gaussianity_score, total_variance_score, mutual_independence_score
        loss, _, _, _ = reconstruction_loss(unmixing_matrix, whitened_gradient)
        loss.backward()
        optimizer.step()
        
        if (iter_idx + 1) % 1000 == 0 or iter_idx == 0:
            print('loss: {}'.format(loss.item()))
            
    unmixing_matrix = unmixing_matrix.detach().to('cpu')
    whitened_gradient = whitened_gradient.detach().to('cpu')
    
    with torch.no_grad():
        lpips_match, lpips_is_positive, mean_lpips = lpips_gpu(unmixing_matrix @ whitened_gradient, selected_val_batch_data.detach().to('cpu'), return_matches=True)
        return mean_lpips

In [6]:
num_of_trials = 200
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=num_of_trials)

[I 2024-01-03 01:05:52,263] A new study created in memory with name: no-name-669b4dbd-e267-40fa-a2e1-1534d3b443fe


Files already downloaded and verified
Files already downloaded and verified
[torch.Size([100, 3072])]
loss: 20.692304611206055
loss: 3.95686936378479
loss: 3.0965869426727295
loss: 2.730005979537964
loss: 2.663930892944336
loss: 2.656079053878784
loss: 2.6527411937713623
loss: 2.6512489318847656
loss: 2.6502811908721924
loss: 2.649935722351074
loss: 2.6496825218200684
loss: 2.649622917175293
loss: 2.6489932537078857
loss: 2.648784637451172
loss: 2.649263381958008
loss: 2.648571014404297
loss: 2.6490323543548584
loss: 2.6490466594696045
loss: 2.6489431858062744
loss: 2.6486141681671143
loss: 2.648634433746338
loss: 2.6485989093780518
loss: 2.6484498977661133
loss: 2.648736000061035
loss: 2.648751735687256
loss: 2.648949384689331


[I 2024-01-03 01:07:03,016] Trial 0 finished with value: 0.18770070688333362 and parameters: {'t_param': 2.641357095835091, 'total_variance_loss_param': 7.752083289424025, 'mutual_independence_loss_param': 2.6123200730451366}. Best is trial 0 with value: 0.18770070688333362.


Files already downloaded and verified
Files already downloaded and verified
[torch.Size([100, 3072])]
loss: 88.57840728759766
loss: 5.951749801635742
loss: 4.2679219245910645
loss: 3.690380811691284
loss: 3.5125253200531006
loss: 3.3413186073303223
loss: 3.1770803928375244
loss: 3.067115306854248
loss: 3.0125327110290527
loss: 2.9881014823913574
loss: 2.9778807163238525
loss: 2.9731533527374268
loss: 2.970639944076538
loss: 2.968412399291992
loss: 2.9677696228027344
loss: 2.9680395126342773
loss: 2.9668946266174316
loss: 2.966327428817749
loss: 2.966384172439575
loss: 2.9659409523010254
loss: 2.966423273086548
loss: 2.9658799171447754
loss: 2.9663994312286377
loss: 2.96614146232605
loss: 2.9659581184387207
loss: 2.96626615524292


[I 2024-01-03 01:08:14,906] Trial 1 finished with value: 0.18858541664667428 and parameters: {'t_param': 4.524926678606968, 'total_variance_loss_param': 5.373425407297111, 'mutual_independence_loss_param': 2.922753035932245}. Best is trial 0 with value: 0.18770070688333362.


...

[I 2024-01-03 03:03:48,160] Trial 97 finished with value: 0.1806982164271176 and parameters: {'t_param': 5.515279430846296, 'total_variance_loss_param': 3.1037700156499373, 'mutual_independence_loss_param': 7.907508895483197}. Best is trial 64 with value: 0.1765547322575003.


Files already downloaded and verified
Files already downloaded and verified
[torch.Size([100, 3072])]
loss: 52.02009201049805
loss: 9.23164176940918
loss: 8.00078296661377
loss: 7.89103889465332
loss: 7.8343281745910645
loss: 7.777996063232422
loss: 7.728933811187744
loss: 7.689977169036865
loss: 7.660122871398926
loss: 7.6373186111450195
loss: 7.62000036239624
loss: 7.606875419616699
loss: 7.596487522125244
loss: 7.58839225769043
loss: 7.582237720489502
loss: 7.5781073570251465
loss: 7.574373722076416
loss: 7.571650505065918
loss: 7.569457530975342
loss: 7.568089008331299
loss: 7.566762447357178
loss: 7.5664496421813965
loss: 7.5655927658081055
loss: 7.564874649047852
loss: 7.564334869384766
loss: 7.563687324523926


[I 2024-01-03 03:04:59,814] Trial 98 finished with value: 0.1815652084769681 and parameters: {'t_param': 2.561989145686771, 'total_variance_loss_param': 2.7967312720226998, 'mutual_independence_loss_param': 7.524262714797694}. Best is trial 64 with value: 0.1765547322575003.


Files already downloaded and verified
Files already downloaded and verified
[torch.Size([100, 3072])]
loss: 76.3044662475586
loss: 11.203654289245605
loss: 9.460253715515137
loss: 9.230082511901855
loss: 9.152280807495117
loss: 9.071755409240723
loss: 8.999773979187012
loss: 8.94204330444336
loss: 8.897899627685547
loss: 8.865029335021973
loss: 8.840436935424805
loss: 8.821882247924805
loss: 8.807705879211426
loss: 8.79702091217041
loss: 8.789021492004395
loss: 8.783023834228516
loss: 8.77878475189209
loss: 8.775469779968262
loss: 8.773209571838379
loss: 8.770548820495605
loss: 8.769055366516113
loss: 8.76834487915039
loss: 8.767245292663574
loss: 8.76657772064209
loss: 8.76618766784668
loss: 8.765893936157227


[I 2024-01-03 03:06:11,355] Trial 99 finished with value: 0.1810831786133349 and parameters: {'t_param': 2.8776750321967266, 'total_variance_loss_param': 3.833025086002568, 'mutual_independence_loss_param': 8.712942583979032}. Best is trial 64 with value: 0.1765547322575003.
